# P2PNet Inference Notebook
This notebook's purpose was to try to load the out of the box P2P model, and later try to use the concept implemented in that paper into the baseline MCNN.
However, due to difficulties on the implementation, and the low expectations we had for this approach, we ended up abandoning it

## 1. Setup

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tthien/shanghaitech")

print("Path to dataset files:", path)

100%|██████████| 333M/333M [00:03<00:00, 92.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/tthien/shanghaitech/versions/1


In [ ]:
!pip install torch torchvision tensorboardX easydict pandas numpy scipy matplotlib Pillow opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.2 MB/s eta 0:00:00


## 2. Configuration

In [ ]:
import os
import torch

# TODO: Change this to the path of your project in Google Drive
#PROJECT_PATH = '/content/drive/MyDrive/CrowdCounting-P2PNet'

import sys
# sys.path.append(PROJECT_PATH)

# Path to the folder containing images
#IMAGE_PATH = os.path.join(PROJECT_PATH, 'images')
IMAGE_PATH = "/root/.cache/kagglehub/datasets/tthien/shanghaitech/versions/1/ShanghaiTech/part_A/train_data/images"

# Path to the folder containing ground truth files
# GT_PATH = os.path.join(PROJECT_PATH, 'gt')
GT_PATH = "/root/.cache/kagglehub/datasets/tthien/shanghaitech/versions/1/ShanghaiTech/part_A/train_data/ground-truth"

# Path to the model weights
#WEIGHTS_PATH = os.path.join(PROJECT_PATH, 'weights/SHTechA.pth')
WEIGHTS_PATH = "/content/SHTechA.pth"
# Device to run the model on
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## 3. Model Definition

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.hub import load_state_dict_from_url
from collections import OrderedDict
import numpy as np
from easydict import EasyDict as edict
from typing import Optional, List
from torch import Tensor

# Misc utils
class NestedTensor(object):
    def __init__(self, tensors, mask: Optional[Tensor]):
        self.tensors = tensors
        self.mask = mask

    def to(self, device):
        # type: (Device) -> NestedTensor # noqa
        cast_tensor = self.tensors.to(device)
        mask = self.mask
        if mask is not None:
            assert mask is not None
            cast_mask = mask.to(device)
        else:
            cast_mask = None
        return NestedTensor(cast_tensor, cast_mask)

    def decompose(self):
        return self.tensors, self.mask

    def __repr__(self):
        return str(self.tensors)


def _max_by_axis_pad(the_list):
    maxes = the_list[0]
    for sublist in the_list[1:]:
        for index, item in enumerate(sublist):
            maxes[index] = max(maxes[index], item)

    block = 128

    for i in range(2):
        maxes[i+1] = ((maxes[i+1] - 1) // block + 1) * block
    return maxes


def nested_tensor_from_tensor_list(tensor_list: List[Tensor]):
    if tensor_list[0].ndim == 3:
        max_size = _max_by_axis_pad([list(img.shape) for img in tensor_list])
        batch_shape = [len(tensor_list)] + max_size
        b, c, h, w = batch_shape
        dtype = tensor_list[0].dtype
        device = tensor_list[0].device
        tensor = torch.zeros(batch_shape, dtype=dtype, device=device)
        mask = torch.ones((b, h, w), dtype=torch.bool, device=device)
        for img, pad_img, m in zip(tensor_list, tensor, mask):
            pad_img[: img.shape[0], : img.shape[1], : img.shape[2]].copy_(img)
            m[: img.shape[1], :img.shape[2]] = False
    else:
        raise ValueError('not supported')
    return NestedTensor(tensor, mask)


# VGG Backbone
model_urls = {
    'vgg16_bn': 'https://download.pytorch.org/models/vgg16_bn-6c64b313.pth',
}


class VGG(nn.Module):

    def __init__(self, features, num_classes=1000, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


def make_layers(cfg, batch_norm=False, sync=False):
    layers = []
    in_channels = 3
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfgs = {
    'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
}


def _vgg(arch, cfg, batch_norm, pretrained, progress, **kwargs):
    if pretrained:
        kwargs['init_weights'] = False
    model = VGG(make_layers(cfgs[cfg], batch_norm=batch_norm), **kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls[arch], progress=progress)
        model.load_state_dict(state_dict)
    return model


def vgg16_bn(pretrained=False, progress=True, **kwargs):
    return _vgg('vgg16_bn', 'D', True, pretrained, progress, **kwargs)


class BackboneBase_VGG(nn.Module):

    def __init__(self, backbone: nn.Module, num_channels: int, name: str, return_interm_layers: bool):
        super().__init__()
        features = list(backbone.features.children())
        if return_interm_layers:
            if name == 'vgg16_bn':
                self.body1 = nn.Sequential(*features[:13])
                self.body2 = nn.Sequential(*features[13:23])
                self.body3 = nn.Sequential(*features[23:33])
                self.body4 = nn.Sequential(*features[33:43])
            else:
                self.body1 = nn.Sequential(*features[:9])
                self.body2 = nn.Sequential(*features[9:16])
                self.body3 = nn.Sequential(*features[16:23])
                self.body4 = nn.Sequential(*features[23:30])
        else:
            if name == 'vgg16_bn':
                self.body = nn.Sequential(*features[:44])  # 16x down-sample
            elif name == 'vgg16':
                self.body = nn.Sequential(*features[:30])  # 16x down-sample
        self.num_channels = num_channels
        self.return_interm_layers = return_interm_layers

    def forward(self, tensor_list: NestedTensor):
        out = []
        if self.return_interm_layers:
            xs = tensor_list.tensors
            for _, layer in enumerate([self.body1, self.body2, self.body3, self.body4]):
                xs = layer(xs)
                out.append(xs)
        else:
            xs = self.body(tensor_list.tensors)
            out.append(xs)
        return out


class Backbone_VGG(BackboneBase_VGG):
    """ResNet backbone with frozen BatchNorm."""
    def __init__(self, name: str, return_interm_layers: bool):
        if name == 'vgg16_bn':
            backbone = vgg16_bn(pretrained=True)
        elif name == 'vgg16':
            backbone = vgg16(pretrained=True)
        num_channels = 512
        super().__init__(backbone, num_channels, name, return_interm_layers)


def build_backbone(args):

    backbone = Backbone_VGG(args.backbone, True)

    return backbone

class RegressionModel(nn.Module):
    def __init__(self, num_features_in, num_anchor_points=4, feature_size=256):
        super(RegressionModel, self).__init__()

        self.conv1 = nn.Conv2d(num_features_in, feature_size, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()

        self.conv3 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act3 = nn.ReLU()

        self.conv4 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act4 = nn.ReLU()

        self.output = nn.Conv2d(feature_size, num_anchor_points * 2, kernel_size=3, padding=1)
    # sub-branch forward
    def forward(self, x):
        out = self.conv1(x)
        out = self.act1(out)

        out = self.conv2(out)
        out = self.act2(out)

        out = self.conv3(out)
        out = self.act3(out)

        out = self.conv4(out)
        out = self.act4(out)

        out = self.output(out)

        out = out.permute(0, 2, 3, 1)

        return out.contiguous().view(out.shape[0], -1, 2)

# the network frmawork of the classification branch
class ClassificationModel(nn.Module):
    def __init__(self, num_features_in, num_anchor_points=4, num_classes=80, prior=0.01, feature_size=256):
        super(ClassificationModel, self).__init__()

        self.num_classes = num_classes
        self.num_anchor_points = num_anchor_points

        self.conv1 = nn.Conv2d(num_features_in, feature_size, kernel_size=3, padding=1)
        self.act1 = nn.ReLU()

        self.conv2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act2 = nn.ReLU()

        self.conv3 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act3 = nn.ReLU()

        self.conv4 = nn.Conv2d(feature_size, feature_size, kernel_size=3, padding=1)
        self.act4 = nn.ReLU()

        self.output = nn.Conv2d(feature_size, num_anchor_points * num_classes, kernel_size=3, padding=1)
        self.output_act = nn.Sigmoid()
    # sub-branch forward
    def forward(self, x):
        out = self.conv1(x)
        out = self.act1(out)

        out = self.conv2(out)
        out = self.act2(out)

        out = self.conv3(out)
        out = self.act3(out)

        out = self.conv4(out)
        out = self.act4(out)

        out = self.output(out)

        out1 = out.permute(0, 2, 3, 1)

        batch_size, width, height, _ = out1.shape

        out2 = out1.view(batch_size, width, height, self.num_anchor_points, self.num_classes)

        return out2.contiguous().view(x.shape[0], -1, self.num_classes)

# generate the reference points in grid layout
def generate_anchor_points(stride=16, row=3, line=3):
    row_step = stride / row
    line_step = stride / line

    shift_x = (np.arange(1, line + 1) - 0.5) * line_step - stride / 2
    shift_y = (np.arange(1, row + 1) - 0.5) * row_step - stride / 2

    shift_x, shift_y = np.meshgrid(shift_x, shift_y)

    anchor_points = np.vstack((
        shift_x.ravel(), shift_y.ravel()
    )).transpose()

    return anchor_points
# shift the meta-anchor to get an acnhor points
def shift(shape, stride, anchor_points):
    shift_x = (np.arange(0, shape[1]) + 0.5) * stride
    shift_y = (np.arange(0, shape[0]) + 0.5) * stride

    shift_x, shift_y = np.meshgrid(shift_x, shift_y)

    shifts = np.vstack((
        shift_x.ravel(), shift_y.ravel()
    )).transpose()

    A = anchor_points.shape[0]
    K = shifts.shape[0]
    all_anchor_points = (anchor_points.reshape((1, A, 2)) + shifts.reshape((1, K, 2)).transpose((1, 0, 2)))
    all_anchor_points = all_anchor_points.reshape((K * A, 2))

    return all_anchor_points

# this class generate all reference points on all pyramid levels
class AnchorPoints(nn.Module):
    def __init__(self, pyramid_levels=None, strides=None, row=3, line=3):
        super(AnchorPoints, self).__init__()

        if pyramid_levels is None:
            self.pyramid_levels = [3, 4, 5, 6, 7]
        else:
            self.pyramid_levels = pyramid_levels

        if strides is None:
            self.strides = [2 ** x for x in self.pyramid_levels]

        self.row = row
        self.line = line

    def forward(self, image):
        image_shape = image.tensors.shape[2:]
        image_shape = np.array(image_shape)
        image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in self.pyramid_levels]

        all_anchor_points = np.zeros((0, 2)).astype(np.float32)
        # get reference points for each level
        for idx, p in enumerate(self.pyramid_levels):
            anchor_points = generate_anchor_points(self.strides[idx], row=self.row, line=self.line)
            shifted_anchor_points = shift(image_shapes[idx], self.strides[idx], anchor_points)
            all_anchor_points = np.append(all_anchor_points, shifted_anchor_points, axis=0)

        all_anchor_points = np.expand_dims(all_anchor_points, axis=0)
        # send reference points to device
        if torch.cuda.is_available():
            return torch.from_numpy(all_anchor_points.astype(np.float32)).cuda()
        else:
            return torch.from_numpy(all_anchor_points.astype(np.float32))

class Decoder(nn.Module):
    def __init__(self, C3_size, C4_size, C5_size, feature_size=256):
        super(Decoder, self).__init__()

        # upsample C5 to get P5 from the FPN paper
        self.P5_1 = nn.Conv2d(C5_size, feature_size, kernel_size=1, stride=1, padding=0)
        self.P5_upsampled = nn.Upsample(scale_factor=2, mode='nearest')
        self.P5_2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=1, padding=1)

        # add P5 elementwise to C4
        self.P4_1 = nn.Conv2d(C4_size, feature_size, kernel_size=1, stride=1, padding=0)
        self.P4_upsampled = nn.Upsample(scale_factor=2, mode='nearest')
        self.P4_2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=1, padding=1)

        # add P4 elementwise to C3
        self.P3_1 = nn.Conv2d(C3_size, feature_size, kernel_size=1, stride=1, padding=0)
        self.P3_2 = nn.Conv2d(feature_size, feature_size, kernel_size=3, stride=1, padding=1)


    def forward(self, inputs):
        C3, C4, C5 = inputs

        P5_x = self.P5_1(C5)
        P5_upsampled_x = self.P5_upsampled(P5_x)
        P5_x = self.P5_2(P5_x)

        P4_x = self.P4_1(C4)
        P4_x = P5_upsampled_x + P4_x
        P4_upsampled_x = self.P4_upsampled(P4_x)
        P4_x = self.P4_2(P4_x)

        P3_x = self.P3_1(C3)
        P3_x = P3_x + P4_upsampled_x
        P3_x = self.P3_2(P3_x)

        return [P3_x, P4_x, P5_x]

# the defenition of the P2PNet model
class P2PNet(nn.Module):
    def __init__(self, backbone, row=2, line=2):
        super().__init__()
        self.backbone = backbone
        self.num_classes = 2
        # the number of all anchor points
        num_anchor_points = row * line

        self.regression = RegressionModel(num_features_in=256, num_anchor_points=num_anchor_points)
        self.classification = ClassificationModel(num_features_in=256, \
                                            num_classes=self.num_classes, \
                                            num_anchor_points=num_anchor_points)

        self.anchor_points = AnchorPoints(pyramid_levels=[4,], row=row, line=line)

        self.fpn = Decoder(256, 512, 512)

    def forward(self, samples: NestedTensor):
        # get the backbone features
        features = self.backbone(samples)
        # forward the feature pyramid
        features_fpn = self.fpn([features[1], features[2], features[3]])

        batch_size = features[0].shape[0]
        # run the regression and classification branch
        regression = self.regression(features_fpn[1]) * 100
        classification = self.classification(features_fpn[1])

        anchor_points = self.anchor_points(samples).repeat(batch_size, 1, 1)
        # decode the points as prediction
        output_coord = regression + anchor_points
        output_class = classification
        out = {'pred_logits': output_class, 'pred_points': output_coord}

        return out

def build(args, training=False):
    # treats persons as a single class
    num_classes = 1

    backbone = build_backbone(args)
    model = P2PNet(backbone, args.row, args.line)
    return model


## 4. Load Model and Weights

In [ ]:
args = edict({
    'backbone': 'vgg16_bn',
    'row': 2,
    'line': 2,
})

model = build(args, training=False)
model.to(DEVICE)
if WEIGHTS_PATH:
    checkpoint = torch.load(WEIGHTS_PATH, map_location=DEVICE)
    model.load_state_dict(checkpoint['model'])
model.eval()

P2PNet(
  (backbone): Backbone_VGG(
    (body1): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ReLU(inplace=True)
      (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ReLU(inplace=True)
    )
    (body2): Sequential(
      (0): MaxPool2d(ke

## 5. Data Loading and Preprocessing

In [ ]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as standard_transforms
from PIL import Image
import cv2
import glob
import math

def load_data(img_path, gt_path):
    img = Image.open(img_path).convert('RGB')

    # Load ground truth points
    points = []
    if os.path.exists(gt_path):
        with open(gt_path) as f_label:
            for line in f_label:
                x, y = map(float , line.strip().split(' '))
                points.append([x, y])
    points = np.array(points)

    return img, points

class CustomDataset(Dataset):
    def __init__(self, image_dir, gt_dir, transform=None):
        self.image_dir = image_dir
        self.gt_dir = gt_dir
        self.transform = transform
        self.image_files = sorted(glob.glob(os.path.join(image_dir, '*.jpg'))) # Assuming .jpg images

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        img_name = os.path.basename(img_path)
        gt_path = os.path.join(self.gt_dir, img_name.replace('.jpg', '.txt')) # Assuming .txt ground truth

        img, points = load_data(img_path, gt_path)

        original_width, original_height = img.size
        # Round the size to be a multiple of 128, similar to run_test.py
        new_width = math.floor(original_width / 128) * 128
        new_height = math.floor(original_height / 128) * 128
        if new_width == 0: new_width = 128
        if new_height == 0: new_height = 128

        img = img.resize((new_width, new_height), Image.LANCZOS)
        if len(points) > 0:
            points[:, 0] = points[:, 0] * (new_width / original_width)
            points[:, 1] = points[:, 1] * (new_height / original_height)

        if self.transform:
            img = self.transform(img)

        target = {'points': torch.from_numpy(points).float(), 'labels': torch.ones(len(points)).long()}

        return img, target

def collate_fn_crowd(batch):
    batch_new = []
    for b_img, b_target in batch:
        # For single images, ensure consistent dimensionality
        if b_img.ndim == 3:
            b_img = b_img.unsqueeze(0)  # Add batch dimension
        # The model expects target points to be a list of dicts, even for batch size 1
        if 'points' in b_target:
            b_target['point'] = b_target.pop('points') # Rename key for compatibility
        if len(b_target['point'].shape) == 1 and b_target['point'].shape[0] == 0:
            # Handle empty points case
            b_target['point'] = torch.empty((0, 2), dtype=torch.float32)
            b_target['labels'] = torch.empty((0,), dtype=torch.int64)
        else:
            # Squeeze to remove batch dimension if it exists and is 1
            b_target['point'] = b_target['point'].squeeze(0) if b_target['point'].ndim == 3 else b_target['point']
            b_target['labels'] = b_target['labels'].squeeze(0) if b_target['labels'].ndim == 2 else b_target['labels']
        batch_new.append((b_img.squeeze(0), b_target)) # Remove initial batch dim for NestedTensor conversion

    batch_imgs, batch_targets = list(zip(*batch_new))

    # nested_tensor_from_tensor_list expects a list of tensors
    batch_imgs_nested = nested_tensor_from_tensor_list(list(batch_imgs))

    return batch_imgs_nested, list(batch_targets)

# Define transformations
transform = standard_transforms.Compose([
    standard_transforms.ToTensor(),
    standard_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create dataset and dataloader
dataset = CustomDataset(IMAGE_PATH, GT_PATH, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn_crowd)

## 6. Testing and Evaluation

In [ ]:
from torchvision.transforms.functional import to_pil_image
import matplotlib.pyplot as plt

mae = 0
mse = 0
results = []

with torch.no_grad():
    for i, (samples, targets) in enumerate(dataloader):
        samples = samples.to(DEVICE)
        outputs = model(samples)

        outputs_scores = torch.nn.functional.softmax(outputs['pred_logits'], -1)[:, :, 1][0]
        outputs_points = outputs['pred_points'][0]

        threshold = 0.5
        # filter the predictions
        points = outputs_points[outputs_scores > threshold].detach().cpu().numpy().tolist()
        predict_cnt = int((outputs_scores > threshold).sum())

        gt_cnt = len(targets[0]['point'])

        mae += abs(predict_cnt - gt_cnt)
        mse += (predict_cnt - gt_cnt) * (predict_cnt - gt_cnt)

        # Denormalize image for visualization
        inv_normalize = standard_transforms.Normalize(
            mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
            std=[1/0.229, 1/0.224, 1/0.225]
        )
        original_img_tensor = inv_normalize(samples.tensors[0].cpu())
        original_img = to_pil_image(original_img_tensor)

        results.append({
            'image': original_img,
            'predicted_points': points,
            'predicted_count': predict_cnt,
            'ground_truth_points': targets[0]['point'].cpu().numpy().tolist(),
            'ground_truth_count': gt_cnt
        })

mae = mae / len(dataloader)
mse = torch.sqrt(mse / len(dataloader))

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")

RuntimeError: The size of tensor a (196608) must match the size of tensor b (49152) at non-singleton dimension 1

## 7. Visualization

In [ ]:
def plot_results(result, image_id=0):
    img_to_draw = np.array(result[image_id]['image'])
    img_to_draw_pred = img_to_draw.copy()
    img_to_draw_gt = img_to_draw.copy()

    # Draw predicted points
    for p in result[image_id]['predicted_points']:
        cv2.circle(img_to_draw_pred, (int(p[0]), int(p[1])), 2, (0, 0, 255), -1)

    # Draw ground truth points
    for p in result[image_id]['ground_truth_points']:
        cv2.circle(img_to_draw_gt, (int(p[0]), int(p[1])), 2, (0, 255, 0), -1)

    fig, axes = plt.subplots(1, 2, figsize=(20, 10))

    axes[0].imshow(img_to_draw_pred)
    axes[0].set_title(f"Predicted Count: {result[image_id]['predicted_count']}")
    axes[0].axis('off')

    axes[1].imshow(img_to_draw_gt)
    axes[1].set_title(f"Ground Truth Count: {result[image_id]['ground_truth_count']}")
    axes[1].axis('off')

    plt.show()

# Example usage: Plot the first image's results
if len(results) > 0:
    plot_results(results, image_id=0)